In [10]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
# print(file)
# print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

In [11]:
dataset_dir_li=[r"./FR/wav/", r"./IT/wav/", r"./SK/wav/"]
output_file_li=[r"FR_output_features.csv", r"IT_output_features.csv", r"SK_output_features.csv"]

In [9]:
from parselmouth.praat import call
import re
import numpy as np

snd = parselmouth.Sound(r"./SK/wav/imitation/SK0102_1_imitation555_1_mono.wav")

# Extract pitch
pitch = snd.to_pitch()
pitch_values = pitch.selected_array['frequency']

mean_pitch = np.mean(pitch_values)
max_pitch = np.max(pitch_values)
print("mean_pitch, max_pitch", mean_pitch, max_pitch)

# Extract intensity
intensity = snd.to_intensity()
mean_intensity = intensity.get_average()
max_intensity = intensity.get_maximum()
print("mean_intensity, max_intensity", mean_intensity, max_intensity)

# Extract jitter and shimmer
pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 600)
jitter = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
print("jitter", jitter)
shimmer = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
print("shimmer", shimmer)

# Extract NHR
harmonicity = snd.to_harmonicity_cc()
match = re.search(r"Average: ([\d\.]+) dB", str(harmonicity))
# If found, extract the value
if match:
    nhr = float(match.group(1))
else:
    nhr=None

print("nhr", nhr)



mean_pitch, max_pitch 52.44447222275884 195.00827236764988
mean_intensity, max_intensity 68.5725947297317 78.96270293537638
jitter 0.027122118771747437
shimmer 0.08819099350790778
nhr 16.5421896


In [12]:
import parselmouth
import os
import csv

def extract_features(audio_file):
    snd = parselmouth.Sound(audio_file)
    
        # Extract pitch
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    
    mean_pitch = np.mean(pitch_values)
    max_pitch = np.max(pitch_values)
    # print("mean_pitch, max_pitch", mean_pitch, max_pitch)
    
    # Extract intensity
    intensity = snd.to_intensity()
    mean_intensity = intensity.get_average()
    max_intensity = intensity.get_maximum()
    # print("mean_intensity, max_intensity", mean_intensity, max_intensity)
    
    # Extract jitter and shimmer
    pointProcess = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 600)
    jitter = parselmouth.praat.call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    # print("jitter", jitter)
    shimmer = parselmouth.praat.call([snd, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    # print("shimmer", shimmer)
    
    # Extract NHR
    harmonicity = snd.to_harmonicity_cc()
    match = re.search(r"Average: ([\d\.]+) dB", str(harmonicity))
    # If found, extract the value
    if match:
        nhr = float(match.group(1))
    else:
        nhr=None
    
    # print("nhr", nhr)
    
    return [mean_pitch, max_pitch, mean_intensity, max_intensity, jitter, shimmer, nhr]


In [6]:
print(extract_features(r"./SK/wav/imitation/SK0102_1_imitation555_1_mono.wav"))

NameError: name 'np' is not defined

In [13]:
def batch_extract_features(dataset_dir, output_csv):
    
    with open(output_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Mean Pitch", "Max Pitch", "Mean Intensity", "Max Intensity", "Jitter", "Shimmer", "NHR"])
        
        for dirpath, dirnames, filenames in os.walk(dataset_dir):
            for audio_file in filenames:
                if audio_file.endswith('.wav'):
                    features = extract_features(os.path.join(dirpath, audio_file))
                    # print("processing", os.path.join(dirpath, audio_file))
                    # print(features)
                    writer.writerow([audio_file] + features)

In [14]:
for i in range(len(dataset_dir_li)):
    print("--Processing--", dataset_dir_li[i], output_file_li[i])
    batch_extract_features(dataset_dir_li[i], output_file_li[i])

batch_extract_features(r"./FR/wav/", r"FR_output_features.csv")

--Processing-- ./FR/wav/ FR_output_features.csv
--Processing-- ./IT/wav/ IT_output_features.csv
--Processing-- ./SK/wav/ SK_output_features.csv
